# Chapter 6: Write

## Question-Answer Generator

The goal of this notebook is to create a model which can generate question and answer pairs about a block of text. This project is based on the [`qgen-workshop` TensorFlow codebase](https://github.com/Maluuba/qgen-workshop). The model consists of two components:

- An RNN which identifies possible question answers from a block of text.

- An encoder-decoder network that generates possible questions that the answers identified by the former model could be for.

An _encoder-decoder_ network is a type of RNN that outputs a new sequence from its input. Some applications of encoder-decoder networks include machine translation, question generation, and text summarization. An encoder-decoder model trains an encoder RNN to encode the input sequence into a vector input for the decoder RNN which outputs a novel sequence from the vector input.

## Question-Answer Dataset

### Step 1: Downloading the data from the Maluuba News QA repository

Below we download and preprocess the data for the model. The data we are using for this notebook is provided by the [Maluuba News QA GitHub repository](https://github.com/Maluuba/newsqa). We use the manual setup instructions with the relevant code below:

In [0]:
# Mount drive.

from google.colab import drive

drive.mount('/content/gdrive/')
base_dir = '/content/gdrive/My Drive/gdl_models/qa/'

In [0]:
!git clone https://github.com/Maluuba/newsqa

Cloning into 'newsqa'...
remote: Enumerating objects: 13, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 132 (delta 4), reused 4 (delta 2), pack-reused 119
Receiving objects: 100% (132/132), 610.20 KiB | 2.45 MiB/s, done.
Resolving deltas: 100% (53/53), done.


For legal reasons, you have to download the data yourself on [Microsoft's website](https://msropendata.com/datasets/939b1042-6402-4697-9c15-7a28de7e1321) and state what you are using the data for. To save myself the trouble of having to reupload the data each time we get a new Colab kernel, 

In [0]:
import os
import subprocess

def load_or_copy(filename):
  """Load a file from Drive or copy it locally into Drive."""
  drive_path = base_dir + filename
  if os.path.isfile(drive_path):
    print('File exists in drive')
    subprocess.call(['cp', drive_path, '.'])
  else:
    print('File exists locally')
    subprocess.call(['cp', filename, drive_path])

In [0]:
load_or_copy('newsqa.tar.gz')

File exists in drive


In [0]:
!mv newsqa.tar.gz newsqa/maluuba/newsqa/newsqa.tar.gz

Now we also download the CNN stories we will use to train the model. You can download the stories [here](https://cs.nyu.edu/~kcho/DMQA/).

In [0]:
load_or_copy('cnn_stories.tgz')

File exists in drive


In [0]:
!mv cnn_stories.tgz newsqa/maluuba/newsqa/cnn_stories.tgz

Now we upload the Java dependencies:

In [0]:
!wget https://nlp.stanford.edu/software/stanford-postagger-2015-12-09.zip

In [0]:
load_or_copy('stanford-postagger-2015-12-09.zip')

File exists in drive


In [0]:
!cp -r stanford-postagger-2015-12-09.zip newsqa/maluuba/newsqa/

Now we run the data processing script in the repository. We run the repository's tests to make sure the data was processed correctly.

In [0]:
!cd newsqa && python2 maluuba/newsqa/data_generator.py

In [0]:
!cd newsqa && python2 -m unittest discover .

[INFO] 2020-05-04 00:25:51,936 - data_processing.py::__init__
Loading dataset from `/content/newsqa/maluuba/newsqa/newsqa-data-v1.csv`...
[INFO] 2020-05-04 00:25:51,936 - data_processing.py::load_combined
Loading data from `/content/newsqa/maluuba/newsqa/newsqa-data-v1.csv`...
[INFO] 2020-05-04 00:25:52,456 - data_processing.py::__init__
Loading stories from `/content/newsqa/maluuba/newsqa/cnn_stories.tgz`...
Getting story texts: 100% 12.7k/12.7k [00:12<00:00, 1.05k stories/s] 
Setting story texts: 100% 120k/120k [00:03<00:00, 37.0k questions/s] 
[INFO] 2020-05-04 00:26:07,792 - data_processing.py::__init__
Done loading dataset.
Checking for possible corruption: 100% 120k/120k [00:01<00:00, 103k questions/s]
.[INFO] 2020-05-04 00:26:09,045 - data_processing.py::dump
Packaging dataset to `/content/newsqa/combined-newsqa-data-v1.json`.
Building json: 100% 120k/120k [00:05<00:00, 21.7k questions/s] 
Checking for possible corruption: 100% 12.7k/12.7k [00:00<00:00, 18.7k stories/s]
Gatherin

Now let's save the data to Drive. Once the code below runs, we will have a copy of the data that'll persist through different Colab kernel sessions.

In [0]:
def upload_to_drive(filepath):
  """Copy a file to Drive."""
  drive_path = base_dir + filepath.split('/')[-1]
  subprocess.call(['cp', filepath, drive_path])

In [0]:
upload_to_drive('newsqa/split_data/train.csv')

In [0]:
upload_to_drive('newsqa/split_data/test.csv')

In [0]:
upload_to_drive('newsqa/split_data/dev.csv')

### Step 2: Download GloVe embeddings

We need to download a set of pretrained word embeddings that were created using the Stanford GloVe (Global Vectors) algorithm, an unsupervised learning algorithm that uses a large set of words.

You can run the code below to download the embeddings.

In [7]:
!mkdir -p glove && wget http://nlp.stanford.edu/data/glove.6B.zip

--2020-05-07 02:55:59--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2020-05-07 02:56:00--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2020-05-07 02:56:00--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6

In [0]:
import zipfile

with zipfile.ZipFile('glove.6B.zip', 'r') as z:
    z.extractall('glove')

I'll upload the 100-dimensional embeddings we'll be using for the model to Drive so I don't have to download the entire zip file again.

In [0]:
upload_to_drive('glove/glove.6B.100d.txt')

Now we are going to trim the embeddings down to only the relevant encodings for our training set are kept.

In [0]:
import csv
from collections import Counter


MAX_DOCUMENT_LENGTH = 200


def tokenize(words):
  """Transform a string of words into uniformly lower case tokens."""
  return [w.lower() for w in words.split(' ')]


def load_data(csv_path):
  """Load the data from the CSV files generated by the Maluuba News QA repository."""
  stories = {}
  with open(csv_path) as f:
    for i, row in enumerate(csv.reader(f)):
      if i == 0:  # Header row.
        continue

      doc_id = row[0]
      existing_stories = stories.setdefault(doc_id, [])

      doc_text = row[1]
      if existing_stories and doc_text == existing_stories[0]['document_text']:
        doc_words = existing_stories[0]['document_words']
      else:
        doc_words = tokenize(doc_text)[:MAX_DOCUMENT_LENGTH]

      q_text = row[2]
      q_words = tokenize(q_text)

      answer = row[3]
      ans_indices = []
      for chunk in answer.split(','):
        start, end = (int(idx) for idx in chunk.split(':'))
        if end < MAX_DOCUMENT_LENGTH:
          ans_indices.extend(range(start, end))
      ans_text = ' '.join(doc_words[i] for i in ans_indices)

      if ans_indices:
        existing_stories.append({
            'document_id': doc_id,
            'document_text': doc_text,
            'document_words': doc_words,
            'answer_text': ans_text,
            'answer_indices': ans_indices,
            'question_text': q_text,
            'question_words': q_words,
        })
  return stories


def trim_embeddings():
  """Trim the Stanford GloVe embeddings to the 10,000 most common words in the training set."""
  doc_counts = Counter()
  q_counts = Counter()

  for data in [load_data(base_dir + 'train.csv').values(),
               load_data(base_dir + 'test.csv').values()]:
    for stories in data:
      if stories:
        doc_counts.update(stories[0]['document_words'])
        for story in stories:
          q_counts.update(story['question_words'])
  
  keep = set()
  for word, _ in q_counts.most_common(5000):
    keep.add(word)
  for word, _ in doc_counts.most_common():
    if len(keep) >= 10000:
      break
    keep.add(word)

  with open(base_dir + 'glove.6B.100d.txt') as f:
    with open(base_dir + 'glove.6B.100d.trimmed.txt', 'w') as f2:
      for line in f:
        if line.split(' ')[0] in keep:
          f2.write(line)

In [0]:
trim_embeddings()

### Step 3: Preprocessing the data for the model

In [0]:
import random


START = '<START>'
END = '<END>'
MAX_BATCH_SIZE = 128


def lookup_word():
  """"""
  pass


def prepare_batch(batch):
  """"""
  ids_to_indices = {}
  doc_ids = []
  doc_text = []
  doc_words = []
  ans_text = []
  ans_indices = []
  q_text = []
  q_input_words = []
  q_output_words = []
  for i, entry in enumerate(batch):
    ids_to_indices.setdefault(entry['document_id'], []).append(i)
    doc_ids.append(entry['document_id'])
    doc_text.append(entry['document_text'])
    doc_words.append(entry['document_words'])
    ans_text.append(entry['answer_text'])
    ans_indices.append(entry['answer_indices'])
    q_text.append(entry['question_text'])
    q_words = entry['question_words']
    q_input_words = [[START] + q_words]
    q_output_words = [q_words + [END]]

  batch_size = len(batch)
  max_doc_len = max((len(doc) for doc in doc_words), default=0)
  max_ans_len = max((len(ans) for ans in ans_indices), default=0)
  max_q_len = max((len(q) for q in q_input_words), default=0)


def process_stories(stories):
  """"""
  batch = []
  vals = list(stories.values())
  random.shuffle(vals)
  for story in vals:
    if len(batch) + len(story) > MAX_BATCH_SIZE:
      yield prepare_batch(batch)
      batch = []
    batch.extend(story)
  if batch:
    yield prepare_batch(batch)